# Using Historical Data and Neural Networks to Predict Fantasy Football Rankings for QB, RB, WR, and TE Positions
### By Max Pickard

Fantasy Football is hard to win. Much of your success through the fantasy season is dependent on how well you draft. For example, one could draft a high tier RB and two very low tier RBs, only to have said player get injured week one, leaving them stuck with low scoring RBs. Or one could draft a player who did well last seasn, only to have that player regress the next season. Using historical data, I plan to build models to predict which players will have the most fantasy football points.  

For this project, we will be focusing only on the QB, RB, WR, and TE positions because these are the main scoring positions. We will also be using the PPR (Points Per Reception) Fantasy Scoring Method to keep everything simple (and since this is the kind of scoring used my fantasy league)


##### Steps:
1. Web Scrap Football Data for QB, RB, WR, and TE positions from Pro-Football Reference 
2. Clean data and organize data into dictionaries with years as keys
3. Build/Train/Test Positonal Models
4. Aggregate all models and rank players based on projected PPR points

## Dependencies 

In [34]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import math
import numpy as np

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

from sklearn.model_selection import train_test_split 
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from matplotlib import pyplot as plt
import warnings

warnings.filterwarnings('ignore')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

## Defining Data Wrangling/Cleaning Functions
**player_data(year)**: Function to Web Scrap Data for a given season

**position_data(stats, pos)**: input data and a specific position as string, returns position stats from given season stats

**PPRs(stats)**: returns dataframe of PPR points from that season

**all_stats(years)**: Creates a dictionary to store multiple dataframes asscoiated with specific year. 
Get stats from range of years and store dfs in dictionary by *id = 'stats.{year}'*

**position_dicts(years, yearStats, pos)**: breaks up larger dictionary (**all_stats**) into smaller dicitonaries of same data by position  
Get positional stats from range of years and store dfs in dictionary by *id = '{position}.{year}'*

**postion_ppr(pos, pos_dict, years)**: create dataframe of PPRs ofr each year in range of years by player in  agiven position

In [2]:
def player_data(year):
    url = "https://www.pro-football-reference.com/years/{}/fantasy.htm".format(year)
    html = urlopen(url)
    soup = BeautifulSoup(html)

    headers = [th.getText() for th in soup.findAll('tr')[1].findAll('th')] #Find the second table row tag, find every table header column within it and extract the html text via the get_text method.
    headers = headers[1:] #Do not need the first (0 index) column header
    
    rows = soup.findAll('tr', class_ = lambda table_rows: table_rows != "thead") #Here we grab all rows that are not classed as table header rows - football reference throws in a table header row everyy 30 rows 
    player_stats = [[td.getText() for td in rows[i].findAll('td')] #get the table data cell text from each table data cell
                    for i in range(len(rows))] #for each row
    player_stats = player_stats[2:]

    stats = pd.DataFrame(player_stats, columns = headers)
    
    stats = stats.replace(r'', 0, regex=True)
    stats['Year'] = year
    
    # fix names, add ProBowl, FirstTeam
    ProBowl = []
    FirstTeam = []
    players = stats['Player']
    for i in range(stats.shape[0]):
        if players[i].find('*') == -1:
            ProBowl.append(0)
        else:
            ProBowl.append(1)
            players[i] = players[i].replace('*', '')

    for i in range(players.size):
        if players[i].find('+') == -1:
            FirstTeam.append(0)
        else:
            FirstTeam.append(1)
            players[i] = players[i].replace('+', '')

    stats['ProBowl'] = ProBowl
    stats['FirstTeam'] = FirstTeam
    stats['Player'] = players
    
    # Differentiate different types of touchdowns
    stats.columns.values[7] = 'PassAtt'
    stats.columns.values[11] = 'RushAtt'
    stats.columns.values[9] = 'PassTD'
    stats.columns.values[14] = 'RushTD'
    stats.columns.values[19] = 'RecTD'
    stats.columns.values[22] = 'TotalTD'
    
    # Drop other scoring methods besides PPR
    stats = stats.drop(columns = ['FantPt', 'DKPt', 'FDPt'])
    
    # Convert all strings to numerical formats
    cols = stats.columns
    
    # Get Columns with Integer Values and Float Values
    intCols = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29]
    floatCols = [13, 18, 25]

    ints = [cols[index] for index in intCols]
    floats = [cols[index] for index in floatCols]

    for i in floats:
        stats = stats.astype({i:"float"})

    for j in ints:
        stats = stats.astype({j:"int"})
        
#     stats.to_csv('{}playerstats.csv'.format(year)) #add your desired path to the function
    print("Player data for the year {} has been created.".format(year))
    return stats
    

In [3]:
def position_data(stats, pos):
    qb = stats[stats['FantPos'] == 'QB'].reset_index()
    rb = stats[stats['FantPos'] == 'RB'].reset_index()
    wr = stats[stats['FantPos'] == 'WR'].reset_index()
    te = stats[stats['FantPos'] == 'TE'].reset_index()
    
    rb['Usage'] = rb['RushAtt'] + rb['Tgt']
    rb['UsageRank'] = rb['Usage'].rank(ascending=False)
    rb['PPRRank'] = rb['PPR'].rank(ascending=False)

    wr['Usage'] = rb['RushAtt'] + rb['Tgt']
    wr['UsageRank'] = rb['Usage'].rank(ascending=False)
    wr['PPRRank'] = rb['PPR'].rank(ascending=False)

    te['Usage'] = rb['RushAtt'] + rb['Tgt']
    te['UsageRank'] = rb['Usage'].rank(ascending=False)
    te['PPRRank'] = rb['PPR'].rank(ascending=False)
    
    if pos == 'qb':
        return qb
    elif pos == 'rb':
        return rb
    elif pos == 'wr':
        return wr
    else:
        return te
    

In [4]:
def PPRs(stats):
    ppr = pd.DataFrame()
    PPRs = stats[['Player','PPR']]
    return PPRs

In [5]:
# Get stats from range of years and store dfs in dictionary by id = 'stats.{year}'
def all_stats(years):
    ids = []

    for j in years:
        ids.append(str(j))

    yearStats = {}

    for name in ids:
        year = int(name)
        yearStats[name] = player_data(year)
    
    return(yearStats)

In [6]:
# Get positional stats from range of years and store dfs in dictionary by id = '{position}.{year}'
def position_dicts(years, yearStats, pos):
    qb_ids = []
    rb_ids = []
    wr_ids = []
    te_ids = []

    qbs={} # QB dicitonary
    rbs={} # RB dicitonary
    wrs={} # WR dicitonary
    tes={} # TE dicitonary

    for k in years:
        qb_ids.append(str(k))
        rb_ids.append(str(k))
        wr_ids.append(str(k))
        te_ids.append(str(k))

    for name in qb_ids:
        year = int(name)
        qbs[name] = position_data(yearStats[str(year)], 'qb') 

    for name in rb_ids:
        year = int(name)
        rbs[name] = position_data(yearStats[str(year)], 'rb') 

    for name in wr_ids:
        year = int(name)
        wrs[name] = position_data(yearStats[str(year)], 'wr') 

    for name in te_ids:
        year = int(name)
        tes[name] = position_data(yearStats[str(year)], 'te') 
    
    if pos == 'qb':
        return qbs
    elif pos == 'rb':
        return rbs
    elif pos == 'wr':
        return wrs
    else:
        return tes

In [7]:
def postion_ppr(pos, pos_dict, years):
    names = []
    Players = pd.DataFrame()
    pos = pd.DataFrame()
    for i in years:
        n = pos_dict[str(i)]['Player'].tolist()
        names = list(set(names + n))
    pos['Player'] = names

    for j in years:
        ppr = PPRs(pos_dict[str(j)])
        ppr[str(j)] = ppr['PPR']
        ppr = ppr.drop(columns = 'PPR')
        pos = pd.merge(pos, ppr, how="outer")
#         pos.fillna('N/A', inplace=True)
        
    return pos.set_index('Player')

In [8]:
# yearStats = all_stats(years)
# qbs = position_dicts(years, yearStats, 'qb')
# qb.ppr = postion_ppr('qb', qbs, years)
# qbs['2017']
# qb.ppr['2017PPR']

## Data Collection/Wrangling
For this project, we will use Fantasy Data from the last decade, (2011, 2021)

In [9]:
years = list(range(2011, 2022))
years

[2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

In [10]:
# Statistics from 2011-2021
decade_stats = all_stats(years)

Player data for the year 2011 has been created.
Player data for the year 2012 has been created.
Player data for the year 2013 has been created.
Player data for the year 2014 has been created.
Player data for the year 2015 has been created.
Player data for the year 2016 has been created.
Player data for the year 2017 has been created.
Player data for the year 2018 has been created.
Player data for the year 2019 has been created.
Player data for the year 2020 has been created.
Player data for the year 2021 has been created.


In [11]:
# Positional Statistics from 2011 to 2021
qbs = position_dicts(years, decade_stats, 'qb')
rbs = position_dicts(years, decade_stats, 'rb')
wrs = position_dicts(years, decade_stats, 'wr')
tes = position_dicts(years, decade_stats, 'te')

In [12]:
# Positional PPRs from 2011-2021
qbs_ppr = postion_ppr('qb', qbs, years)
rbs_ppr = postion_ppr('rb', rbs, years)
wrs_ppr = postion_ppr('wr', wrs, years)
tes_ppr = postion_ppr('te', tes, years)

# Functions for Training Models and Hyperparameter Tuning
We will be creating 4 models, 1 model for each position. Each model will be trained using all the data from a previous season, and will be tested with the PPR data from the season (i.e. using the 2011 RB data to train the model to predict the 2012 RB PPR, and evaulating the results against the actual 2012 RB PPR). Our predictions of the PPR will be returned in a normalized form, and instead of converting the output to PPR, given that we are more interested in who will be the best vs how many point they score, and since it is easier to rank them based on the output.

To perform hyperparameter tuning, we use grid search. The functions and methodology used in the grid search can be found here: https://datagraphi.com/blog/post/2019/12/17/how-to-find-the-optimum-number-of-hidden-layers-and-nodes-in-a-neural-network-model.  

All credit goes to the author. The functions were rewritten to use learning rates and take in a custom imput size for the first layer based on the number of columns in the training dataset. This is because the QB dataset does not have the *Usage* and *UsageRank* attributes like the other datasets

In [201]:
def FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes):
    layers = []
    
    nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
    nodes = first_layer_nodes
    for i in range(1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    
    return layers

In [235]:
def createmodel(n_layers, first_layer_nodes, last_layer_nodes, activation_func, loss_func, learning_rate, input_shape):
    model = Sequential()
    n_nodes = FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes)
    for i in range(1, n_layers):
        if i==1:
            model.add(Dense(first_layer_nodes, input_dim=input_shape, activation=activation_func))
        else:
            model.add(Dense(n_nodes[i-1], activation=activation_func))
            
    #Finally, the output layer should have a single node in binary classification
    model.add(Dense(1, activation=activation_func))
    model.compile(optimizer=keras.optimizers.SGD(lr=learning_rate), loss=loss_func, metrics = ["mse"]) #note: metrics could also be 'mse'
    
    return model

##Wrap model into scikit-learn
model =  KerasClassifier(build_fn=createmodel, verbose = False)

In [274]:
# Function to create and run Neural Network, returns test prediction, y_test, and the model itself
def network(pos, pos_ppr, train_years):
    # Setup Data
    # X, players index will correspond with each other, making it each to match with one another
    # We keep PPRnext and Player together so that when we randomize dataframe we can keep the two together
    X = pd.DataFrame()
    players = pd.DataFrame()
    for i in train_years:
        x = pos[str(i)]
        y= pos_ppr[str(i+1)]

        data = pd.merge(x, y, on="Player")
        data = data.rename(columns={data.columns[-1]: 'PPRnext'}) # Create dependent variables
        data = data.dropna() # We only use predict PPR for players who played the next season
        data = data.drop(columns=['Year'])

    # Create Training and Testing data
    X = X.append(data.iloc[:,4:35 ])
    players = players.append(data[['Player', 'PPRnext']])

    # Normalize Columns
    y = players['PPRnext']
    y_mean = players['PPRnext'].mean()
    y_std = players['PPRnext'].std()

    X_norm=(X-X.mean())/X.std()
    X_norm = X_norm.fillna(0) #Change NaN to 0.0
    Y_norm=(y-y_mean)/y_std
    
    # Split Data
    x_train, x_test, y_train, y_test = train_test_split(X_norm, Y_norm, test_size=0.2, random_state=42)
    
    X_train = x_train.to_numpy()
    Y_train = y_train.to_numpy()
    
    activation_funcs = ['relu', 'tanh', 'logistic'] 
    loss_funcs = ['mean_squared_error']
    param_grid = dict(n_layers=[2, 3], first_layer_nodes = [200,100,50], last_layer_nodes = [4],  
                      activation_func = activation_funcs, loss_func = loss_funcs, batch_size = [100], epochs = [100], 
                      learning_rate= [0.01, 0.001, 0.0001], input_shape =[X_train.shape[1]])
    grid = GridSearchCV(estimator = model, param_grid = param_grid)

    grid.fit(X_train,y_train)
    
    opt = grid.best_params_
    
    # Create Neural Network
    NN = MLPRegressor(max_iter=opt['epochs'], activation = opt['activation_func'], batch_size = opt['batch_size'], 
                       hidden_layer_sizes = (opt['first_layer_nodes'], opt['last_layer_nodes']), learning_rate = 'constant',
                       learning_rate_init = opt['learning_rate'], solver= 'lbfgs')
    
    # Train/Fit the MLPRegressor NN
    NN.fit(x_train,y_train)

    # Predict using test input
    NN_pred = NN.predict(x_test)
    
    return NN_pred, y_test, NN

## RB Model

In [275]:
# years to iterate through for training
years = list(range(2011, 2021))

In [276]:
pred, rb_y_test, rb_NN = network(rbs, rbs_ppr, years)
# opt = network(rbs, rbs_ppr, years)
# Compute Error
print("MSE: ", mean_squared_error(rb_y_test, pred))
print("MAE: ", mean_absolute_error(rb_y_test, pred))

MSE:  0.5753448136709955
MAE:  0.5806137322053081


## QB Model

In [277]:
pred, qb_y_test, qb_NN = network(qbs, qbs_ppr, years)
# Compute Error
print("MSE: ", mean_squared_error(qb_y_test, pred))
print("MAE: ", mean_absolute_error(qb_y_test, pred))

MSE:  0.07491443850964752
MAE:  0.16837818808716282


## WR Model

In [304]:
pred, wr_y_test, wr_NN = network(wrs, wrs_ppr, years)
# Compute Error
print("MSE: ", mean_squared_error(wr_y_test, pred))
print("MAE: ", mean_absolute_error(wr_y_test, pred))

MSE:  1.2930134017459851
MAE:  0.9389546479666326


## TE Model

In [279]:
pred, te_y_test, te_NN = network(tes, tes_ppr, years)
# Compute Error
print("MSE: ", mean_squared_error(te_y_test, pred))
print("MAE: ", mean_absolute_error(te_y_test, pred))

MSE:  1.464068249387438
MAE:  0.90520013597468


# Predicting 2021 Rankings with 2020 Stats
The following function allows for a position dictionary, postion ppr dictionary, postion neural network, and year to use for predicting the following years ranking, and returns a dataframe showing the previosu rank, predicted rank, actual rank, and the difference. It the difference is negative, then the neural network over-predicted the rank, and if its positive then the network under-predicted the rank


In [280]:
def predict_test(pos, pos_ppr, pos_NN, year):
    x = pos[str(year)]
    y= pos_ppr[str(year+1)]

    data = pd.merge(x, y, on="Player")
    data = data.rename(columns={data.columns[-1]: 'PPRnext'}) # Create dependent variables
    data = data.dropna() # We only use predict PPR for players who played the next season
    data = data.drop(columns=['Year'])

    # # Create Training and Testing data
    ranks = pos[str(year+1)][['Player','PosRank']]
    X = data.iloc[:,4:35 ]
    players = data[['Player']]

    X_norm=(X-X.mean())/X.std()

    pred =pos_NN.predict(X_norm.fillna(0))

    # preds = convert_PPR(pred, y_std, y_mean)

    players['Predicted'] = pred
    players['PrevRank'] = pos[str(year-1)]['PosRank']
    players = players.sort_values(by=['Predicted'], ascending = False)
    players = players.reset_index()
    players = players.drop(columns = ['index'])
    players['PredRank'] = players.index+1
    players = players.merge(ranks, on='Player')
    players = players.rename(columns = {'PosRank': 'ActRank'})
    players['RankDiff'] = players['PredRank'] - players['ActRank']
    return players.head(10)
#     return(X.shape[1])


## 2021 Rankings Predictions with 2020 Data

In [305]:
pred2021rbs = predict_test(rbs, rbs_ppr, rb_NN, 2020)
pred2021qbs = predict_test(qbs, qbs_ppr, qb_NN, 2020)
pred2021wrs = predict_test(wrs, wrs_ppr, wr_NN, 2020)
pred2021tes = predict_test(tes, tes_ppr, te_NN, 2020)

In [306]:
pred2021rbs 

,Player,Predicted,PrevRank,PredRank,ActRank,RankDiff
0,Jonathan Taylor,3.512917,4.0,1,1,0
1,Ezekiel Elliott,2.288078,11.0,2,6,-4
2,James Conner,2.094136,26.0,3,5,-2
3,Leonard Fournette,2.069539,39.0,4,11,-7
4,Alvin Kamara,1.819107,2.0,5,9,-4
5,Antonio Gibson,1.749430,14.0,6,10,-4
6,Josh Jacobs,1.725108,8.0,7,16,-9
7,Austin Ekeler,1.622197,35.0,8,2,6
8,Nick Chubb,1.600010,9.0,9,7,2
9,Damien Harris,1.548907,45.0,10,8,2


In [307]:
pred2021qbs

,Player,Predicted,PrevRank,PredRank,ActRank,RankDiff
0,Justin Herbert,1.937995,9.0,1,2,-1
1,Tom Brady,1.886589,8.0,2,3,-1
2,Patrick Mahomes,1.789400,4.0,3,4,-1
3,Aaron Rodgers,1.568760,2.0,4,5,-1
4,Matthew Stafford,1.540091,16.0,5,6,-1
5,Dak Prescott,1.473863,31.0,6,7,-1
6,Joe Burrow,1.420393,26.0,7,8,-1
7,Jalen Hurts,1.409871,35.0,8,9,-1
8,Ryan Tannehill,1.395225,7.0,9,12,-3
9,Kyler Murray,1.315617,3.0,10,10,0


In [308]:
pred2021wrs 

,Player,Predicted,PrevRank,PredRank,ActRank,RankDiff
0,Davante Adams,1.866993,1,1,5,-4
1,Keenan Allen,1.866993,18,2,15,-13
2,Stefon Diggs,1.866993,3,3,8,-5
3,D.K. Metcalf,1.866993,5,4,10,-6
4,Justin Jefferson,1.866993,6,5,4,1
5,Tyreek Hill,1.866993,2,6,7,-1
6,A.J. Brown,1.866993,9,7,32,-25
7,DeAndre Hopkins,1.866993,10,8,39,-31
8,Cooper Kupp,1.866993,34,9,1,8
9,Tee Higgins,1.866993,28,10,17,-7


In [309]:
pred2021tes

,Player,Predicted,PrevRank,PredRank,ActRank,RankDiff
0,Mark Andrews,3.603559,4,1,1,0
1,Dalton Schultz,2.158996,17,2,3,-1
2,Rob Gronkowski,1.556006,8,3,5,-2
3,Hunter Henry,1.481385,15,4,7,-3
4,Dawson Knox,1.479304,33,5,6,-1
5,Mike Gesicki,1.477279,6,6,11,-5
6,Dallas Goedert,1.472192,21,7,8,-1
7,Tyler Conklin,1.071428,55,8,19,-11
8,Harrison Bryant,1.042266,40,9,31,-22
9,Darren Waller,1.011086,2,10,18,-8


## Observations
For the RB position, it correct predicted that Johnathan Taylor would be the top RB. Otherwise it was only off be a margin of 2 spots for most of the top 10. It incorrectly predicted that Josh Jacobs and Leonard Fournette would be in the top 10, but Fournette was number 11, and Jacobs was still within the top 20.  

For the QB position, it was almost spot on for the top 10, missing only Josh Allen. Most QBs were overanked by only 1 position, except fot Ryan Tanihill, who was ranked 12 in reality.  
  
For the TE/WR class, the network correctly predicted Cooper Kupp and Mark Andrews would be ranked number 1 in their respective positions. 

The WR rankings were not the most accurate, which would make sensebecause the WR position has a lot of players, allowing for a lot more variability.
  
Rankings for the TE position were not too far off. The network overranked almost all TEs, by usually only a few spots. But all of them were within the top 20.

# 2022 Rankings Predictions with 2021 Data

In [310]:
def pred2022(pos, pos_NN):
    x = pos[str(2021)]

    # Create Training and Testing data
    X = x.iloc[:,4:35 ]
    X_norm=(X-X.mean())/X.std()
    players = x[['Player', 'PPR']]

    pred = pos_NN.predict(X_norm.fillna(0))

    # preds = convert_PPR(pred, X['PPR'].std(), X['PPR'].mean())

    players['Predicted'] = pred
    players = players.sort_values(by=['Predicted'], ascending = False)
    players['PrevRank'] = pos[str(2021)]['PosRank']
    players = players.reset_index()
    players = players.drop(columns=['index'])
    players['PredRank'] = players.index +1
    return players.head(10)

In [311]:
rb2022 = pred2022(rbs, rb_NN)
qb2022 = pred2022(qbs, qb_NN)
wr2022 = pred2022(wrs, wr_NN)
te2022 = pred2022(tes, te_NN)

In [312]:
rb2022

,Player,PPR,Predicted,PrevRank,PredRank
0,Jonathan Taylor,373.1,4.494115,1,1
1,David Montgomery,195.0,3.278394,20,2
2,Michael Carter,154.4,2.737199,30,3
3,James Conner,257.7,2.563474,5,4
4,James Robinson,173.9,2.531353,25,5
5,Josh Jacobs,226.0,1.678107,16,6
6,Joe Mixon,287.9,1.671196,3,7
7,Chase Edmonds,143.3,1.620085,37,8
8,Najee Harris,300.7,1.598264,4,9
9,Antonio Gibson,229.1,1.570865,10,10


In [313]:
qb2022

,Player,PPR,Predicted,PrevRank,PredRank
0,Aaron Rodgers,333.3,6.882199,5,1
1,Josh Allen,402.6,1.170589,1,2
2,Justin Herbert,380.8,0.967278,2,3
3,Patrick Mahomes,361.7,0.759466,4,4
4,Kirk Cousins,300.3,0.624693,11,5
5,Tom Brady,374.7,0.615607,3,6
6,Dak Prescott,320.6,0.608084,7,7
7,Matthew Stafford,329.7,0.556980,6,8
8,Kyler Murray,300.5,0.497613,10,9
9,Joe Burrow,314.2,0.341152,8,10


In [314]:
wr2022

,Player,PPR,Predicted,PrevRank,PredRank
0,Cooper Kupp,439.5,1.866993,1,1
1,Chase Claypool,166.6,1.866993,36,2
2,Marquise Brown,226.3,1.866993,25,3
3,Amari Cooper,202.5,1.866993,26,4
4,Adam Thielen,199.8,1.866993,27,5
5,Christian Kirk,207.6,1.866993,28,6
6,Kendrick Bourne,180.5,1.866993,29,7
7,Van Jefferson,168.2,1.866993,31,8
8,Tyler Boyd,183.8,1.866993,33,9
9,K.J. Osborn,158.5,1.866993,35,10


In [316]:
te2022

,Player,PPR,Predicted,PrevRank,PredRank
0,Kyle Pitts,176.6,6.141593,9,1
1,George Kittle,198.0,5.057739,4,2
2,Travis Kelce,262.8,4.783997,2,3
3,Noah Fant,159.0,3.861843,13,4
4,Rob Gronkowski,171.2,3.757939,5,5
5,Tyler Higbee,147.0,3.480237,14,6
6,Dallas Goedert,165.0,3.319837,8,7
7,Tyler Conklin,138.3,3.146217,19,8
8,Darren Waller,133.5,3.058547,18,9
9,Dalton Schultz,208.8,2.859258,3,10


# Conclusion
The predictions for the 2022 rankings for the RB, QB, and TE positons seems like they will be fairly likely, while the Wide reciever predictions seem somewhat outlandish, as it is predicting many players to jump up by roughly 20 spots. It seems unlikely but as the WR position is very large, it is not completely out of the picture. It is important to remeber that this neural netwrok does not account for off-season moves, new coachs/schemes, or rookies. It it purely based on base statistics from previous games. Overall, I do believe there is a fair amount of accuracy in the model, specifically in the QB category.